In [1]:
import sys as sys 
sys.path.append('..')

import numpy as np 
import pandas as pd
from ModelTools.Model.Model import Model


In [2]:
n = 500
rng = np.random.default_rng(0)
x1 = rng.normal(size=n)
x2 = rng.normal(size=n)
y = rng.normal(loc=np.sin(x1) + np.cos(x2) + np.sin(x1) * np.cos(x2),scale=1)
df = pd.DataFrame(data={'x1':x1,'x2':x2,'y':y,'ts':pd.date_range(start='2000-01-01',periods=n,freq='min')})

# print(df)

m = Model(data=df,col_x=['x1','x2'],col_y='y',col_ts='ts',ts_freq='min')
m.fit()

100%|██████████| 8/8 [00:05<00:00,  1.40it/s]


In [3]:
m.Metric.plot_metric_scatter(type='pca')

alt.VConcatChart(...)

In [7]:
from ModelTools.tools.Pca import Pca 
Pca(data=m.Metric.get_metric(type='eval')).plot_bio()

                     index       PC1       PC2       PC3       PC4       PC5  \
0        Pred_inter_sp_OLS -2.306539  0.352799 -0.261658 -0.082244  0.060383   
1     Pred_inter_sp_std_EN -2.126915  0.325260 -0.300866 -0.124237 -0.051795   
2  Pred_inter_sp_std_HUBER -2.015154  0.252659 -0.330657  0.174476 -0.009803   
3     Pred_inter_sp_std_QR -1.196052  1.026718  0.900485  0.056492 -0.001415   
4            Pred_poly_OLS  0.983985 -1.485599  0.213425 -0.043408  0.001679   
5         Pred_poly_std_EN  0.652973 -1.601771  0.280316 -0.032110 -0.002591   
6      Pred_poly_std_HUBER  2.094979 -0.731292 -0.404161  0.093947  0.003384   
7         Pred_poly_std_QR  3.912722  1.861227 -0.096883 -0.042915  0.000157   

            PC6        R2       MAE       MBE      MAPE       MSE      MAXE  
0  1.931830e-16  0.540140  0.809117  0.037584  1.152536  1.046664  2.617499  
1  5.115967e-16  0.537800  0.808549  0.035229  1.238493  1.051989  2.629314  
2 -7.882371e-16  0.540352  0.813001  0.033088

alt.VConcatChart(...)